In [1]:
import numpy as np
import pandas as pd
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Done')

Solving environment: done

# All requested packages already installed.

Done


First I installed the libraries that I thought i might need

In [60]:
!pip install lxml

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)

In [61]:
toronto = tables[0]

In [62]:
toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


By using pd.read_html I was able to import the table from wikipedia into a dataframe (tables[0]) 

In [63]:
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.rename(columns={"Postal Code":"PostalCode"},inplace = True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [64]:
toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [65]:
toronto.shape

(103, 3)

I made the dataframe toronto, using the datafram tables[0] so that it was more streamlined in name (It actually means something instead of an arbitrary value) and then dropped the non assigned boroughs. This then creates a dataframe where we have the neighbourhoods and boroughs of each postcode.

In [71]:
lonlat = pd.read_csv(r'Geospatial_Coordinates.csv')

None of the Geocoders seemed to be working, I therefore used the CSV file to create a new dataframe of the Longatude, Latitude and Postal code

In [91]:
toronto_data = toronto.merge(lonlat, left_on = 'PostalCode', right_on = 'Postal Code', left_index = False , right_index = False)

In [96]:
toronto_data.drop(columns=['Postal Code'], inplace=True)

Using pd.merge I merged the two dataframes into a new dataframe, toronto_data and then dropped the second Postal Code column, creating the df shown below.

In [97]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
